In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
data = pd.read_csv('./../data/heart.csv')

In [28]:
target = 'target'

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [56]:
data[target].value_counts()

1    526
0    499
Name: target, dtype: int64

In [30]:
feat_cols = list(set(data.columns) - set([target]))

In [31]:
from CrossingBuilder import build_crossings, build_crossings_prototypes
from crossing_functions import crossing_product
from scipy.stats import pointbiserialr as pcorr

In [44]:
depth = 1

In [45]:
prototypes = build_crossings_prototypes(data.drop([target], axis=1), depth)

In [46]:
prototypes

[['age', 'age'],
 ['age', 'sex'],
 ['age', 'cp'],
 ['age', 'trestbps'],
 ['age', 'chol'],
 ['age', 'fbs'],
 ['age', 'restecg'],
 ['age', 'thalach'],
 ['age', 'exang'],
 ['age', 'oldpeak'],
 ['age', 'slope'],
 ['age', 'ca'],
 ['age', 'thal'],
 ['sex', 'sex'],
 ['cp', 'sex'],
 ['sex', 'trestbps'],
 ['chol', 'sex'],
 ['fbs', 'sex'],
 ['restecg', 'sex'],
 ['sex', 'thalach'],
 ['exang', 'sex'],
 ['oldpeak', 'sex'],
 ['sex', 'slope'],
 ['ca', 'sex'],
 ['sex', 'thal'],
 ['cp', 'cp'],
 ['cp', 'trestbps'],
 ['chol', 'cp'],
 ['cp', 'fbs'],
 ['cp', 'restecg'],
 ['cp', 'thalach'],
 ['cp', 'exang'],
 ['cp', 'oldpeak'],
 ['cp', 'slope'],
 ['ca', 'cp'],
 ['cp', 'thal'],
 ['trestbps', 'trestbps'],
 ['chol', 'trestbps'],
 ['fbs', 'trestbps'],
 ['restecg', 'trestbps'],
 ['thalach', 'trestbps'],
 ['exang', 'trestbps'],
 ['oldpeak', 'trestbps'],
 ['slope', 'trestbps'],
 ['ca', 'trestbps'],
 ['thal', 'trestbps'],
 ['chol', 'chol'],
 ['chol', 'fbs'],
 ['chol', 'restecg'],
 ['chol', 'thalach'],
 ['chol', 'ex

In [47]:
from importances import get_point_biserial_corrs

In [48]:
orig_imp = get_point_biserial_corrs(data, target)

In [49]:
crossed_data, cross_info = build_crossings(data, prototypes, crossing_product, orig_imp)

In [50]:
crossed_imp = abs(crossed_data.corr()[target]).sort_values()

In [51]:
pd.DataFrame(crossed_imp)

,target
restecg_X_sex,0.019229
fbs_X_thalach,0.019388
fbs_X_sex,0.031167
fbs_X_slope,0.038021
fbs_X_restecg,0.038749
...,...
exang_X_trestbps,0.445097
cp_X_slope,0.449473
cp_X_thalach,0.454156
oldpeak_X_thal,0.457514


In [52]:
crossed_full_info = pd.DataFrame(crossed_imp)
crossed_full_info['importance'] = crossed_full_info['target']
crossed_full_info = crossed_full_info.drop('target', axis=1)
crossed_full_info['rank'] = pd.Series(cross_info)
crossed_full_info = crossed_full_info.drop('target', axis=0)

In [53]:
crossed_full_info

,importance,rank
restecg_X_sex,0.019229,0.279501
fbs_X_thalach,0.019388,0.422895
fbs_X_sex,0.031167,0.279501
fbs_X_slope,0.038021,0.345512
fbs_X_restecg,0.038749,0.134468
...,...,...
exang_X_thal,0.444305,0.438029
exang_X_trestbps,0.445097,0.438029
cp_X_slope,0.449473,0.434854
cp_X_thalach,0.454156,0.434854


In [54]:
dataset_name = "heart"

In [55]:
crossed_full_info.to_csv(f'../data/output/crossed_imp_{dataset_name}_d{depth}.csv')

In [27]:
pd.DataFrame(orig_imp, columns=['importance']).to_csv(f'../data/output/orig_imp_{dataset_name}.csv', index=True)